This code iteratively solves for the correct ride and roll rate using the method described in section 16 of RCVD

In [2]:
import numpy as np
import pandas as pd
import pickle

# Importing Engine Data from Excel using PANDAS
cbf650r_engine_data_file_path = "C:/Users/maxwe/Downloads/FSAE/2023-2024 Car/Repo/engine_data.xlsx"
cbf650r_engine_data = pd.read_excel(cbf650r_engine_data_file_path)

# Importing Car Model from car_model using PICKLE
with open('C:/Users/maxwe/Downloads/FSAE/2023-2024 Car/Repo/car_model.pkl', 'rb') as f:
    car_var = pickle.load(f)

# total weight of car (minus driver) (lbm)
W_T = car_var['W_T']
# weight bias, if less than 0.5, then the rear of the car will have more weight, if more than 0.5, then the front will have more weight
W_bias = car_var['W_bias']
# length of wheelbase (in)
l = car_var['l']
# vertical center of gravity (in)
h = car_var['h']
# setting pi as a number
pi = 3.14159
# tire grip limit (G's)
A_y = car_var['tire_a']
# front track width (in)
t_f = car_var['t_f']
# rear track width (in)
t_r = car_var['t_r']
# CG to roll axis (in)
H = car_var['H']
# distance between CG and front track (in)
a = car_var['a']
# distance between CG and rear track (in)
b = car_var['b']
# front roll axis height (in)
z_rf = car_var['z_rf']
# rear roll axis height (in)
z_rr = car_var['z_rr']
# Tire rates, camper from 0-4 deg, (lb/in)
T_r = car_var['T_r']


W_f = W_bias*W_T # lbs, weight over front wheels
W_r = (1-W_bias)*W_T# lbs, weight over rear wheels
W_1 = W_f/2 #lbs, front left
W_2 = W_f/2 #lbs, front right
W_3 = W_r/2 #lbs, rear left
W_4 = W_r/2 #lbs, rear right


K_RF = 167 # lb/in, initial ride rates
K_RR = 162

y_F_ideal = 0.65 # This is the ideal wheel travel at maximum lateral acceleration 
y_R_ideal = 0.65

y_F = 1 # We start with a wheel travel displacement above ideal so the while loop runs properly
y_R = 1

while y_R > y_R_ideal:

    K_rollF = 12*K_RF*t_f**2/2 # lb*ft/rad, roll rate
    K_rollR = 12*K_RR*t_r**2/2

    freq_f = 1/(2*np.pi)*np.sqrt(K_RF*12*32.2/W_2) # natrual frequency of ride rate
    freq_r = 1/(2*np.pi)*np.sqrt(K_RR*12*32.2/W_4)

    W_F = A_y*W_T/t_f*(H*K_rollF/(K_rollF+K_rollR)+b/l*z_rf) # weight transfer for front and rear tracks
    W_R = A_y*W_T/t_r*(H*K_rollR/(K_rollF+K_rollR)+a/l*z_rr)

    y_F = W_F/K_RF # wheel travel at maximum lateral acceleration
    y_R = W_R/K_RR

    K_RF = W_F/y_F_ideal # New ride rate value
    K_RR = W_R/y_R_ideal



print(f"The front ride rate is {K_RF} lb/in, the rear ride rate is {K_RR} lb/in\n")
print(f"the front wheel travel at {A_y} G's is {y_F} in, and the rear wheel travel is {y_R} in\n")
print(f"The front natural frequency is {freq_f} Hz, the rear natural frequency is {freq_r} Hz\n")
print(f"The front roll rate is {K_rollF} lb*ft/rad, and the rear roll rate is {K_rollR} lb*ft/rad\n")
print(f"The front roll rate is {K_rollF*pi/180} lb*ft/deg, and the rear roll rate is {K_rollR*pi/180} lb*ft/deg\n")
print(f"The front weight transfer is {W_F} lb, and the rear weight transfer is {W_R} lb")




The front ride rate is 212.15283983903907 lb/in, the rear ride rate is 280.2156185334611 lb/in

the front wheel travel at 1.4 G's is 0.6499999999999998 in, and the rear wheel travel is 0.65 in

The front natural frequency is 3.3069926113579307 Hz, the rear natural frequency is 3.854211021003956 Hz

The front roll rate is 3160689.7952244333 lb*ft/rad, and the rear roll rate is 3976052.6597339646 lb*ft/rad

The front roll rate is 55164.3969654396 lb*ft/deg, and the rear roll rate is 69395.15152940903 lb*ft/deg

The front weight transfer is 137.8993458953754 lb, and the rear weight transfer is 182.14015204674973 lb


In [11]:
# Creating Initial Dataframe from Slip Angle vs. Lateral Force Data

data_h_ia4_50_ = {'SA':h_ia4_50_sa_avg, 'FY':h_ia4_50_fy_avg}
data_h_ia4_100_ = {'SA':h_ia4_100_sa_avg, 'FY':h_ia4_100_fy_avg}
data_h_ia4_150_ = {'SA':h_ia4_150_sa_avg, 'FY':h_ia4_150_fy_avg}
data_h_ia4_250_ = {'SA':h_ia4_250_sa_avg, 'FY':h_ia4_250_fy_avg}
data_h_ia4_350_ = {'SA':h_ia4_350_sa_avg, 'FY':h_ia4_350_fy_avg}

df_h_ia4_50 = pd.DataFrame(data_h_ia4_50_)
df_h_ia4_100 = pd.DataFrame(data_h_ia4_100_)
df_h_ia4_150 = pd.DataFrame(data_h_ia4_150_)
df_h_ia4_250 = pd.DataFrame(data_h_ia4_250_)
df_h_ia4_350 = pd.DataFrame(data_h_ia4_350_)

# Sorting and Averaging Slip Angle by 0.1 increments

for i in [x/10 for x in range(-130, 130, 1)]:
    rows = df_h_ia4_50.loc[((df_h_ia4_50['SA']) > i-0.05) & ((df_h_ia4_50['SA']) < i + 0.05)]
    df_h_ia4_50.loc[i] = [i, rows['FY'].mean()]

for i in [x/10 for x in range(-130, 130, 1)]:
    rows = df_h_ia4_100.loc[((df_h_ia4_100['SA']) > i-0.05) & ((df_h_ia4_100['SA']) < i + 0.05)]
    df_h_ia4_100.loc[i] = [i, rows['FY'].mean()]

for i in [x/10 for x in range(-130, 130, 1)]:
    rows = df_h_ia4_150.loc[((df_h_ia4_150['SA']) > i-0.05) & ((df_h_ia4_150['SA']) < i + 0.05)]
    df_h_ia4_150.loc[i] = [i, rows['FY'].mean()]

for i in [x/10 for x in range(-130, 130, 1)]:
    rows = df_h_ia4_250.loc[((df_h_ia4_250['SA']) > i-0.05) & ((df_h_ia4_250['SA']) < i + 0.05)]
    df_h_ia4_250.loc[i] = [i, rows['FY'].mean()]

for i in [x/10 for x in range(-130, 130, 1)]:
    rows = df_h_ia4_350.loc[((df_h_ia4_350['SA']) > i-0.05) & ((df_h_ia4_350['SA']) < i + 0.05)]
    df_h_ia4_350.loc[i] = [i, rows['FY'].mean()]

# Removing unecessary data lines (Hoosier R25B)

sa_ia1_50 = df_h_ia4_50['SA']
for i in df_h_ia4_50.index:   
    if i != sa_ia1_50[i]:
        df_h_ia4_50 = df_h_ia4_50.drop([i])
df_h_ia4_50 = df_h_ia4_50.sort_values(by='SA')

sa_ia1_100 = df_h_ia4_100['SA']
for i in df_h_ia4_100.index:   
    if i != sa_ia1_100[i]:
        df_h_ia4_100 = df_h_ia4_100.drop([i])
df_h_ia4_100 = df_h_ia4_100.sort_values(by='SA')

sa_ia1_150 = df_h_ia4_150['SA']
for i in df_h_ia4_150.index:   
    if i != sa_ia1_150[i]:
        df_h_ia4_150 = df_h_ia4_150.drop([i])
df_h_ia4_150 = df_h_ia4_150.sort_values(by='SA')

sa_ia1_250 = df_h_ia4_250['SA']
for i in df_h_ia4_250.index:   
    if i != sa_ia1_250[i]:
        df_h_ia4_250 = df_h_ia4_250.drop([i])
df_h_ia4_250 = df_h_ia4_250.sort_values(by='SA')

sa_ia1_350 = df_h_ia4_350['SA']
for i in df_h_ia4_350.index:   
    if i != sa_ia1_350[i]:
        df_h_ia4_350 = df_h_ia4_350.drop([i])
df_h_ia4_350 = df_h_ia4_350.sort_values(by='SA')

zeros = np.zeros(len(df_h_ia4_350.index))

# Creating Master Dataframe

data_h_ia4_mstr = {'SA':df_h_ia4_50['SA'], '0':zeros, '50':df_h_ia4_50['FY'], '100':df_h_ia4_100['FY'],'150':df_h_ia4_150['FY'], '250':df_h_ia4_250['FY'], '350':df_h_ia4_350['FY']}
df_h_ia4_mstr = pd.DataFrame(data_h_ia4_mstr)
df_h_ia4_mstr = df_h_ia4_mstr.dropna()

# Creating multiple Fz values

# Converting Dataframre columns into list
col_lst = df_h_ia4_mstr.columns

# converting dataframe index into a list
indx_lst = []
for i in df_h_ia4_mstr.index:
    indx_lst.append(i)

# data list that will be used to creat master master dataframe
data = []

# Creating in-between values for Fz

# Loop for each slip angle value, ie. -12.7, -12.6, ... , 12.5, 12.6
for i in range(len(indx_lst)):
    lst = []
    # range(1,6) is 0, 50, 100, 150, 250, 350
    for j in range(1,6):
        # slope is dFy/dFz, coincedentally the coefficient of friction. Different slope for different interval, ie. 50-100
        slope = (df_h_ia4_mstr.iloc[i,j+1]-df_h_ia4_mstr.iloc[i,j])/(float(col_lst[j+1])-float(col_lst[j]))
        # Loop for calculating each individual Fy for value for ech Fz value
        for k in range(int(col_lst[j+1])-int(col_lst[j])):
            lst.append(df_h_ia4_mstr.iloc[i,j]+slope*k)
    data.append(lst)

# creating list for columns for dataframe
columns = []
for i in range(350):
    i = float(i)
    columns.append(i)

# Creating ultimate Fy dataframe!
df_h_ia4_fy = pd.DataFrame(data, columns=columns, index=indx_lst)

display(df_h_ia4_fy)


[-1.31000000e+01 -1.30000000e+01 -1.29000000e+01 -1.28000000e+01
 -1.27000000e+01 -1.26000000e+01 -1.25000000e+01 -1.24000000e+01
 -1.23000000e+01 -1.22000000e+01 -1.21000000e+01 -1.20000000e+01
 -1.19000000e+01 -1.18000000e+01 -1.17000000e+01 -1.16000000e+01
 -1.15000000e+01 -1.14000000e+01 -1.13000000e+01 -1.12000000e+01
 -1.11000000e+01 -1.10000000e+01 -1.09000000e+01 -1.08000000e+01
 -1.07000000e+01 -1.06000000e+01 -1.05000000e+01 -1.04000000e+01
 -1.03000000e+01 -1.02000000e+01 -1.01000000e+01 -1.00000000e+01
 -9.90000000e+00 -9.80000000e+00 -9.70000000e+00 -9.60000000e+00
 -9.50000000e+00 -9.40000000e+00 -9.30000000e+00 -9.20000000e+00
 -9.10000000e+00 -9.00000000e+00 -8.90000000e+00 -8.80000000e+00
 -8.70000000e+00 -8.60000000e+00 -8.50000000e+00 -8.40000000e+00
 -8.30000000e+00 -8.20000000e+00 -8.10000000e+00 -8.00000000e+00
 -7.90000000e+00 -7.80000000e+00 -7.70000000e+00 -7.60000000e+00
 -7.50000000e+00 -7.40000000e+00 -7.30000000e+00 -7.20000000e+00
 -7.10000000e+00 -7.00000